In [1]:
import rasterio
import rasterio.features
import numpy as np
from rasterio.transform import from_origin
from scipy.ndimage import uniform_filter
import geopandas as gpd


def create_raster_from_points_with_smoothing(gdf, reference_raster_path, column, pixel_size, output_raster_path, smoothing_window=3):
    """
    Create a raster file based on point data from a GeoDataFrame and apply a focal mean smoothing filter.

    Parameters:
    - gdf: GeoDataFrame containing points with a specified column of values.
    - reference_raster_path: Path to the reference raster to define extent.
    - column: Column in the GeoDataFrame to rasterize (e.g., 'diff').
    - pixel_size: Desired pixel size for the new raster.
    - output_raster_path: Path to save the output raster.
    - smoothing_window: Size of the window for the focal mean filter (default 3x3).

    Returns:
    - None
    """
    # Open the reference raster to get bounds and CRS
    with rasterio.open(reference_raster_path) as ref:
        bounds = ref.bounds
        crs = ref.crs
    
    # Calculate raster dimensions based on pixel size
    width = int((bounds.right - bounds.left) / pixel_size)
    height = int((bounds.top - bounds.bottom) / pixel_size)
    
    # Define transform
    transform = from_origin(bounds.left, bounds.top, pixel_size, pixel_size)
    
    # Create a numpy array to hold the raster data (initialize with NaN)
    raster = np.full((height, width), np.nan, dtype=np.float32)
    
    # Rasterize points
    for _, row in gdf.iterrows():
        # Get the point coordinates and value
        x, y = row.geometry.x, row.geometry.y
        value = row[column]
        
        # Convert geographic coordinates to raster indices
        col = int((x - bounds.left) / pixel_size)
        row = int((bounds.top - y) / pixel_size)
        
        # Add value to raster if within bounds
        if 0 <= col < width and 0 <= row < height:
            if np.isnan(raster[row, col]):
                raster[row, col] = value
            else:
                raster[row, col] = (raster[row, col] + value) / 2  # Compute the mean
    
    # Replace NaN with a no-data value
    nodata = -9999
    raster = np.where(np.isnan(raster), nodata, raster)
    
    # Apply focal mean smoothing filter (ignoring no-data values)
    valid_mask = raster != nodata
    smoothed_raster = uniform_filter(np.where(valid_mask, raster, 0), size=smoothing_window)
    weight_mask = uniform_filter(valid_mask.astype(float), size=smoothing_window)
    raster_smoothed = np.where(weight_mask > 0, smoothed_raster / weight_mask, nodata)
    
    # Write the raster to file
    with rasterio.open(
        output_raster_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=raster_smoothed.dtype,
        crs=crs,
        transform=transform,
        nodata=nodata
    ) as dst:
        dst.write(raster_smoothed, 1)

In [2]:
import os
import geopandas as gpd
from pathlib import Path

# Function for batch processing
def process_shapefiles(input_folder, reference_raster_path, output_folder, column="h_mean", pixel_size=100, smoothing_window=3):
    """
    Processes all shapefiles in a folder, rasterizes them using a static reference raster, and saves the output rasters.

    Parameters:
        input_folder (str): Path to the folder containing shapefiles.
        reference_raster_path (str): Path to the static reference raster.
        output_folder (str): Path to the folder where output rasters will be saved.
        column (str): The column to rasterize from the shapefile.
        pixel_size (int): Desired pixel size for rasterization.
        smoothing_window (int): Smoothing window size.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Loop through all shapefiles in the input folder
    for shapefile_path in Path(input_folder).glob("*.shp"):
        try:
            # Read shapefile
            gdf = gpd.read_file(shapefile_path, driver="ESRI Shapefile")

            # Define output raster filename (same name but with .tif extension)
            output_raster_path = os.path.join(output_folder, shapefile_path.stem + ".tif")

            # Call the rasterization function
            create_raster_from_points_with_smoothing(
                gdf=gdf,
                reference_raster_path=reference_raster_path,
                column=column,
                pixel_size=pixel_size,
                output_raster_path=output_raster_path,
                smoothing_window=smoothing_window
            )

            print(f"Processed: {shapefile_path.name} -> {output_raster_path}")

        except Exception as e:
            print(f"Error processing {shapefile_path.name}: {e}")

# Example usage
process_shapefiles(
    input_folder=r"D:\2410_Grounding-Line\IS2_tracks\284",  # Folder containing input shapefiles
    reference_raster_path=r"D:\2312_Storstrommen\AeroDEM1978\AeroDEM1978_EPSG3413_100m_storstrommen.tif",  # Static reference raster
    output_folder=r"D:\2410_Grounding-Line\IS2_tracks\284\rasterized",  # Folder to save output rasters
    column="h_mean",
    pixel_size=100,
    smoothing_window=3
)


C:\Users\gjm501\AppData\Local\Temp\ipykernel_5344\2771753445.py:64: RuntimeWarning: invalid value encountered in divide
  raster_smoothed = np.where(weight_mask > 0, smoothed_raster / weight_mask, nodata)


Processed: atl06_sr_10_11012021.shp -> D:\2410_Grounding-Line\IS2_tracks\284\rasterized\atl06_sr_10_11012021.tif
Processed: atl06_sr_12_12072021.shp -> D:\2410_Grounding-Line\IS2_tracks\284\rasterized\atl06_sr_12_12072021.tif
Processed: atl06_sr_13_11102021.shp -> D:\2410_Grounding-Line\IS2_tracks\284\rasterized\atl06_sr_13_11102021.tif
Processed: atl06_sr_14_09012022.shp -> D:\2410_Grounding-Line\IS2_tracks\284\rasterized\atl06_sr_14_09012022.tif
Processed: atl06_sr_18_08012023.shp -> D:\2410_Grounding-Line\IS2_tracks\284\rasterized\atl06_sr_18_08012023.tif
Processed: atl06_sr_19_08042023.shp -> D:\2410_Grounding-Line\IS2_tracks\284\rasterized\atl06_sr_19_08042023.tif
Processed: atl06_sr_21_07102023.shp -> D:\2410_Grounding-Line\IS2_tracks\284\rasterized\atl06_sr_21_07102023.tif
Processed: atl06_sr_22_06012024.shp -> D:\2410_Grounding-Line\IS2_tracks\284\rasterized\atl06_sr_22_06012024.tif
Processed: atl06_sr_23_06042024.shp -> D:\2410_Grounding-Line\IS2_tracks\284\rasterized\atl06_sr

In [ ]:
import rasterio
import rasterio.features
import numpy as np
from rasterio.transform import from_origin
from scipy.ndimage import uniform_filter
import geopandas as gpd


def create_raster_from_points_with_smoothing(gdf, reference_raster_path, column, pixel_size, output_raster_path, smoothing_window=3):
    """
    Create a raster file based on point data from a GeoDataFrame and apply a focal mean smoothing filter.

    Parameters:
    - gdf: GeoDataFrame containing points with a specified column of values.
    - reference_raster_path: Path to the reference raster to define extent.
    - column: Column in the GeoDataFrame to rasterize (e.g., 'diff').
    - pixel_size: Desired pixel size for the new raster.
    - output_raster_path: Path to save the output raster.
    - smoothing_window: Size of the window for the focal mean filter (default 3x3).

    Returns:
    - None
    """
    # Open the reference raster to get bounds and CRS
    with rasterio.open(reference_raster_path) as ref:
        bounds = ref.bounds
        crs = ref.crs
    
    # Calculate raster dimensions based on pixel size
    width = int((bounds.right - bounds.left) / pixel_size)
    height = int((bounds.top - bounds.bottom) / pixel_size)
    
    # Define transform
    transform = from_origin(bounds.left, bounds.top, pixel_size, pixel_size)
    
    # Create a numpy array to hold the raster data (initialize with NaN)
    raster = np.full((height, width), np.nan, dtype=np.float32)
    
    # Rasterize points
    for _, row in gdf.iterrows():
        # Get the point coordinates and value
        x, y = row.geometry.x, row.geometry.y
        value = row[column]
        
        # Convert geographic coordinates to raster indices
        col = int((x - bounds.left) / pixel_size)
        row = int((bounds.top - y) / pixel_size)
        
        # Add value to raster if within bounds
        if 0 <= col < width and 0 <= row < height:
            if np.isnan(raster[row, col]):
                raster[row, col] = value
            else:
                raster[row, col] = (raster[row, col] + value) / 2  # Compute the mean
    
    # Replace NaN with a no-data value
    nodata = -9999
    raster = np.where(np.isnan(raster), nodata, raster)
    
    # Apply focal mean smoothing filter (ignoring no-data values)
    valid_mask = raster != nodata
    smoothed_raster = uniform_filter(np.where(valid_mask, raster, 0), size=smoothing_window)
    weight_mask = uniform_filter(valid_mask.astype(float), size=smoothing_window)
    raster_smoothed = np.where(weight_mask > 0, smoothed_raster / weight_mask, nodata)
    
    # Write the raster to file
    with rasterio.open(
        output_raster_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=raster_smoothed.dtype,
        crs=crs,
        transform=transform,
        nodata=nodata
    ) as dst:
        dst.write(raster_smoothed, 1)


atl06_sr_filtered = gpd.read_file(r'D:\2410_Grounding-Line\hardangervidda_140120_HR.shp', driver='ESRI Shapefile')
# Example usage
create_raster_from_points_with_smoothing(
    gdf=atl06_sr_filtered,
    reference_raster_path=r'D:\2312_Storstrommen\AeroDEM1978\AeroDEM1978_EPSG3413_100m_storstrommen.tif',
    column="h_mean",
    pixel_size=100,  # Define desired pixel size
    output_raster_path=r'D:\2410_Grounding-Line\Storstrom_140120_HR.tif',
    smoothing_window=3  # Define smoothing window size (5x5)
)